In [1]:
"""
    TODO
    1. 명칭 및 구조 통일화
    2. 그래프 및 그림 추가
    3. 피처 추가
"""

'\n    TODO\n    1. 명칭 및 구조 통일화\n    2. 그래프 및 그림 추가\n    3. 피처 추가\n'

In [2]:
# pip install optuna
# !pip install optuna-integration

In [3]:
# pip install torch pandas numpy scikit-learn xgboost optuna matplotlib

In [4]:
# !pip uninstall torch torchvision torchaudio
# !pip cache purge
# !pip install torch torchvision torchaudio

In [5]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import torch
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
from xgboost.callback import EvaluationMonitor
import optuna
import matplotlib.pyplot as plt

c:\Users\syyun\Desktop\Programming\Hackerton\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
try:
    CUDA = torch.cuda.is_available()
except:
    CUDA = False

print(f"CUDA available: {CUDA}")

CUDA available: False


In [7]:
# -----------------------------
# 1) 경로 설정
# -----------------------------
TRAIN_FP = Path('../open/train/train.csv')
TEST_DIR = Path('../open/test')
SAMPLE_FP = Path('../open/sample_submission.csv')

if not TRAIN_FP.exists() or not TEST_DIR.exists() or not SAMPLE_FP.exists():
    print("Path error: directory or file does not exist. Please check the paths.")
    print(f"Expected file path: {TRAIN_FP}")
    exit()
else:
    print("File paths are valid.")

File paths are valid.


In [8]:
# -----------------------------
# 2) 데이터 로드 및 전처리
# -----------------------------
print("Data loading and preprocessing...")

train = pd.read_csv('../open/train/train.csv')
train["영업일자"] = pd.to_datetime(train["영업일자"])

print("Outlier handling using IQR...")

# 분포 비교를 위해 사본 보관
train_raw = train.copy()

# 이상치 처리 함수
def handle_outliers_iqr(df_group):
    non_zero_sales = df_group[df_group["매출수량"] > 0]["매출수량"]
    if len(non_zero_sales) < 5:
        return df_group

    q1, q3 = non_zero_sales.quantile(0.25), non_zero_sales.quantile(0.75)
    iqr = q3 - q1

    # 이상치 기준
    lower_bound = max(0, q1 - 1.5 * iqr)
    upper_bound = q3 + 1.5 * iqr

    df_group["매출수량"] = np.clip(df_group["매출수량"], lower_bound, upper_bound)
    return df_group

train = train.groupby("영업장명_메뉴명", group_keys=False).apply(handle_outliers_iqr)

print("Outlier handling complete")

tests = {}
for i in range(10):
    name = f"TEST_{i:02d}"
    df = pd.read_csv(TEST_DIR / f"{name}.csv")
    df["영업일자"] = pd.to_datetime(df["영업일자"])
    tests[name] = df
    print(f"[{name}] shape: {df.shape} | 날짜: {df['영업일자'].min().date()} ~ {df['영업일자'].max().date()}")

Data loading and preprocessing...
Outlier handling using IQR...
Outlier handling complete
[TEST_00] shape: (5404, 3) | 날짜: 2024-06-16 ~ 2024-07-13
[TEST_01] shape: (5404, 3) | 날짜: 2024-07-21 ~ 2024-08-17
[TEST_02] shape: (5404, 3) | 날짜: 2024-08-25 ~ 2024-09-21
[TEST_03] shape: (5404, 3) | 날짜: 2024-09-29 ~ 2024-10-26
[TEST_04] shape: (5404, 3) | 날짜: 2024-11-03 ~ 2024-11-30
[TEST_05] shape: (5404, 3) | 날짜: 2024-12-08 ~ 2025-01-04
[TEST_06] shape: (5404, 3) | 날짜: 2025-01-12 ~ 2025-02-08
[TEST_07] shape: (5404, 3) | 날짜: 2025-02-16 ~ 2025-03-15
[TEST_08] shape: (5404, 3) | 날짜: 2025-03-23 ~ 2025-04-19
[TEST_09] shape: (5404, 3) | 날짜: 2025-04-27 ~ 2025-05-24


In [9]:
# -----------------------------
# 라이브러리 임포트
# -----------------------------
import pandas as pd
import numpy as np
from pathlib import Path

# -----------------------------
# 1) 경로 설정
# -----------------------------
TRAIN_FP = Path('../open/train/train.csv')
TEST_DIR = Path('../open/test')
SAMPLE_FP = Path('../open/sample_submission.csv')

# -----------------------------
# 2) 데이터 로드 및 전처리
# -----------------------------
print("Data loading and preprocessing...")

train = pd.read_csv(TRAIN_FP)
train["영업일자"] = pd.to_datetime(train["영업일자"])

print("Outlier handling using IQR...")

train_raw = train.copy()  # 분포 비교용 원본 저장

# 이상치 처리 함수
def handle_outliers_iqr(df_group):
    non_zero_sales = df_group[df_group["매출수량"] > 0]["매출수량"]
    if len(non_zero_sales) < 5:
        return df_group

    q1, q3 = non_zero_sales.quantile(0.25), non_zero_sales.quantile(0.75)
    iqr = q3 - q1

    lower_bound = max(0, q1 - 1.5 * iqr)
    upper_bound = q3 + 1.5 * iqr

    df_group.loc[:, "매출수량"] = np.clip(df_group["매출수량"], lower_bound, upper_bound)
    return df_group

# 그룹별 이상치 처리
train = train.groupby("영업장명_메뉴명", group_keys=False).apply(handle_outliers_iqr)

print("✅ Outlier handling complete")

# Test 데이터 로드
tests = {}
for i in range(10):
    name = f"TEST_{i:02d}"
    df = pd.read_csv(TEST_DIR / f"{name}.csv")
    df["영업일자"] = pd.to_datetime(df["영업일자"])
    tests[name] = df
    print(f"[{name}] shape: {df.shape} | 날짜: {df['영업일자'].min().date()} ~ {df['영업일자'].max().date()}")


Data loading and preprocessing...
Outlier handling using IQR...
✅ Outlier handling complete
[TEST_00] shape: (5404, 3) | 날짜: 2024-06-16 ~ 2024-07-13
[TEST_01] shape: (5404, 3) | 날짜: 2024-07-21 ~ 2024-08-17
[TEST_02] shape: (5404, 3) | 날짜: 2024-08-25 ~ 2024-09-21
[TEST_03] shape: (5404, 3) | 날짜: 2024-09-29 ~ 2024-10-26
[TEST_04] shape: (5404, 3) | 날짜: 2024-11-03 ~ 2024-11-30
[TEST_05] shape: (5404, 3) | 날짜: 2024-12-08 ~ 2025-01-04
[TEST_06] shape: (5404, 3) | 날짜: 2025-01-12 ~ 2025-02-08
[TEST_07] shape: (5404, 3) | 날짜: 2025-02-16 ~ 2025-03-15
[TEST_08] shape: (5404, 3) | 날짜: 2025-03-23 ~ 2025-04-19
[TEST_09] shape: (5404, 3) | 날짜: 2025-04-27 ~ 2025-05-24


In [10]:
# -----------------------------
# 3) 피처 엔지니어링
# -----------------------------
print("Feature engineering...")

encoder = LabelEncoder()
train["item_id"] = encoder.fit_transform(train["영업장명_메뉴명"])

# 영업일자에서 날짜 관련 피처 생성
def make_date_feats(df):
    out = df.copy()
    out["year"], out["month"], out["day"], out["weekday"] = (
        out["영업일자"].dt.year,
        out["영업일자"].dt.month,
        out["영업일자"].dt.day,
        out["영업일자"].dt.weekday,
    )
    out["is_weekend"] = (
        out["weekday"].isin([5, 6]).astype(int)
    )

    # 주기적 특성 변환
    out["month_sin"], out["month_cos"] = np.sin(
        2 * np.pi * out["month"] / 12.0
    ), np.cos(2 * np.pi * out["month"] / 12.0)
    out["wday_sin"], out["wday_cos"] = np.sin(2 * np.pi * out["weekday"] / 7.0), np.cos(
        2 * np.pi * out["weekday"] / 7.0
    )
    return out

# 영업일자 피처 생성 및 정렬
train = make_date_feats(train)
train = train.sort_values(["item_id", "영업일자"])

for lag in [1, 7, 14, 28]:
    train[f"lag_{lag}"] = train.groupby("item_id")["매출수량"].shift(lag)

g = train.groupby("item_id")["매출수량"]
train["roll_mean_7"] = g.shift(1).rolling(7).mean()
train["roll_mean_14"] = g.shift(1).rolling(14).mean()
train["roll_std_7"] = g.shift(1).rolling(7).std()

train = train.dropna()
print("Feature engineering complete")

feature_cols = [
    "year",
    "month",
    "day",
    "weekday",
    "is_weekend",
    "month_sin",
    "month_cos",
    "wday_sin",
    "wday_cos",
    "item_id",
    "lag_1",
    "lag_7",
    "lag_14",
    "lag_28",
    "roll_mean_7",
    "roll_mean_14",
    "roll_std_7",
]

# =========================================
# [패치] 2024 가중치 + 특정 매장 가중치 + 인덱스 정렬
# =========================================

# 1) 기본 가중치
train['weight'] = 1.0

# 2) 특정 매장 가중치 (담하/미라시아) — 필요 시 값 조절
train.loc[train['영업장명_메뉴명'].str.contains('담하|미라시아', na=False), 'weight'] *= 3

# 3) 2024년 가중치
USE_RAMP_2024 = False  # True: 2024 안에서 연초→연말 선형증가(정교), False: 고정배수(간단)

mask_2024 = train['year'] == 2024

# Anchor
# 고정 배수(간단): 2024 전체 ×1.5 (원하면 1.2~2.0 사이로 조절)
train.loc[mask_2024, 'weight'] *= 1.45

# 4) 값이 0인 경우 가중치 감소
train.loc[train['매출수량'] == 0, 'weight'] *= 0.25

# (선택) 가중치 분포 확인
print("▶ weight 분포")
print(train['weight'].value_counts().sort_index())

# 5) 최종 학습 입력 구성 + 인덱스 정렬(중요)
X = train[feature_cols].reset_index(drop=True)
y = train["매출수량"].astype(float).reset_index(drop=True)
weights = train["weight"].astype(float).reset_index(drop=True)


Feature engineering...
Feature engineering complete
▶ weight 분포
weight
0.2500    20530
0.3625     9498
0.7500    12065
1.0000    19910
1.0875     5380
1.4500    10542
3.0000    12536
4.3500     6811
Name: count, dtype: int64


C:\Users\syyun\AppData\Local\Temp\ipykernel_14388\3274158213.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['weight'] = 1.0


In [11]:
# -----------------------------
# 4) Optuna를 이용한 하이퍼파라미터 튜닝 (가중치/재현성 반영 버전)
# -----------------------------
print("Optuna hyperparameter tuning...")

# 재현성 강화를 위한 전역 시드 (필요 시 상단 공통 영역으로 이동해도 무방)
import random
random.seed(42)
np.random.seed(42)

tscv = TimeSeriesSplit(n_splits=5)
splits = list(tscv.split(X))

# 검증 점수에 가중치 반영 여부 (True: 가중 RMSE, False: 비가중 RMSE)
USE_WEIGHTED_EVAL = True

def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "tree_method": "hist",
        # GPU의 미세한 비결정성 회피가 필요하면 'cpu' 권장
        "device": "cuda" if CUDA else "cpu",
        "seed": 42,
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    rmses = []
    for tr_idx, va_idx in splits:
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]
        w_tr, w_va = weights.iloc[tr_idx], weights.iloc[va_idx]  # ← 2024/특정매장 가중치 반영

        dtr = xgb.DMatrix(X_tr, label=y_tr, weight=w_tr)
        if USE_WEIGHTED_EVAL:
            dva = xgb.DMatrix(X_va, label=y_va, weight=w_va)  # 평가도 가중
        else:
            dva = xgb.DMatrix(X_va, label=y_va)               # 평가 비가중

        cb = optuna.integration.XGBoostPruningCallback(trial, "val-rmse")
        model = xgb.train(
            params,
            dtr,
            num_boost_round=1000,
            evals=[(dva, "val")],
            early_stopping_rounds=50,
            callbacks=[cb],
            verbose_eval=False,
        )
        rmses.append(float(model.best_score))

    return float(np.mean(rmses))


# Optuna Sampler에 시드 고정(탐색 경로 재현성)
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
    sampler=sampler,
)

warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    message=r"The reported value is ignored because this `step` \d+ is already reported\."
)

study.optimize(objective, n_trials=150)

print(f"Best hyperparameter: {study.best_params}")
print(f"Best RMSE: {study.best_value}")


[I 2025-08-24 21:46:20,748] A new study created in memory with name: no-name-b705d21c-2727-4d9a-af8a-62f56697ce4f


Optuna hyperparameter tuning...


[I 2025-08-24 21:46:23,417] Trial 0 finished with value: 25.85267337227491 and parameters: {'max_depth': 7, 'learning_rate': 0.2536999076681772, 'subsample': 0.892797576724562, 'colsample_bytree': 0.8394633936788146, 'gamma': 1.77071686435378e-07, 'lambda': 1.7699302940633311e-07, 'alpha': 2.9152036385288193e-08}. Best is trial 0 with value: 25.85267337227491.
[I 2025-08-24 21:46:33,106] Trial 1 finished with value: 27.012028527948377 and parameters: {'max_depth': 11, 'learning_rate': 0.07725378389307355, 'subsample': 0.8832290311184181, 'colsample_bytree': 0.608233797718321, 'gamma': 0.574485163632042, 'lambda': 0.04566054873446119, 'alpha': 4.997040685255803e-07}. Best is trial 0 with value: 25.85267337227491.
[I 2025-08-24 21:46:39,107] Trial 2 finished with value: 24.715334648590826 and parameters: {'max_depth': 5, 'learning_rate': 0.018659959624904916, 'subsample': 0.7216968971838151, 'colsample_bytree': 0.8099025726528951, 'gamma': 2.85469785779718e-05, 'lambda': 2.13714073163729

Best hyperparameter: {'max_depth': 5, 'learning_rate': 0.11445424740515175, 'subsample': 0.7523840346576761, 'colsample_bytree': 0.8844934445010932, 'gamma': 2.063615382380159e-05, 'lambda': 0.00010747087773482173, 'alpha': 0.025807405198685764}
Best RMSE: 23.84409384008459


In [12]:
# ==============================================================================
# 5) 랜덤 윈도우를 이용한 앙상블 모델 학습 (수정된 부분)
# ==============================================================================
print("Ensemble model training with random windows...")

# --- 하이퍼파라미터 설정 ---
N_MODELS = 10  # 앙상블에 사용할 모델 개수
WINDOW_SIZE_DAYS = 180  # 학습에 사용할 1개 윈도우의 기간 (일)
VALIDATION_SIZE_DAYS = 7 # 윈도우 바로 다음 검증 기간 (일)
# -------------------------

# Optuna로 찾은 최적 파라미터 사용
best_params = study.best_params
best_params.update(
    {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "tree_method": "hist",
        "device": "cuda" if CUDA else "cpu",
        "seed": 42
    }
)

models = []
min_date = train['영업일자'].min()
# 학습 윈도우와 검증 윈도우를 만들 수 있는 마지막 날짜
max_start_date = train['영업일자'].max() - pd.Timedelta(days=WINDOW_SIZE_DAYS + VALIDATION_SIZE_DAYS)

for i in range(N_MODELS):
    print(f"--- Training model {i+1}/{N_MODELS} ---")

    # 1. 랜덤 윈도우 생성
    # 전체 기간 내에서 랜덤한 시작일 선택
    days_range = (max_start_date - min_date).days
    random_days = np.random.randint(0, days_range)
    window_start_date = min_date + pd.Timedelta(days=random_days)
    window_end_date = window_start_date + pd.Timedelta(days=WINDOW_SIZE_DAYS)
    validation_end_date = window_end_date + pd.Timedelta(days=VALIDATION_SIZE_DAYS)

    print(f"  Window: {window_start_date.date()} ~ {window_end_date.date()} | Validation: {window_end_date.date()} ~ {validation_end_date.date()}")


    # 2. 윈도우에 해당하는 데이터 추출 및 피처 엔지니어링
    # 주의: lag, rolling 피처는 윈도우 이전 데이터까지 포함해서 계산해야 올바릅니다.
    data_for_features = train[train['영업일자'] <= validation_end_date].copy()

    # 피처 재생성
    for lag in [1, 7, 14, 28]:
        data_for_features[f"lag_{lag}"] = data_for_features.groupby("item_id")["매출수량"].shift(lag)

    g = data_for_features.groupby("item_id")["매출수량"]
    data_for_features["roll_mean_7"], data_for_features["roll_mean_14"], data_for_features["roll_std_7"] = (
        g.shift(1).rolling(7).mean(),
        g.shift(1).rolling(14).mean(),
        g.shift(1).rolling(7).std(),
    )
    data_for_features = data_for_features.dropna() # 피처 생성으로 인한 NA 제거

    # 3. 학습/검증 데이터 분리
    train_window_df = data_for_features[data_for_features['영업일자'].between(window_start_date, window_end_date)]
    valid_window_df = data_for_features[data_for_features['영업일자'] > window_end_date]

    # 4. DMatrix 생성 (가중치 적용)
    X_tr = train_window_df[feature_cols]
    y_tr = train_window_df["매출수량"]
    w_tr = train_window_df["weight"] # 기존 가중치 로직은 train 데이터프레임에 이미 적용됨

    X_va = valid_window_df[feature_cols]
    y_va = valid_window_df["매출수량"]

    dtr = xgb.DMatrix(X_tr, label=y_tr, weight=w_tr)
    dva = xgb.DMatrix(X_va, label=y_va)

    # 5. 모델 학습
    model = xgb.train(
        best_params,
        dtr,
        num_boost_round=5000,
        evals=[(dva, "val")],
        early_stopping_rounds=100,
        verbose_eval=False,
    )

    print(f"  Best iteration: {model.best_iteration}, Best RMSE: {model.best_score:.4f}")
    models.append(model)

print("\n✅ Ensemble model training complete.")

Ensemble model training with random windows...
--- Training model 1/10 ---
  Window: 2023-05-11 ~ 2023-11-07 | Validation: 2023-11-07 ~ 2023-11-14
  Best iteration: 14, Best RMSE: 17.5607
--- Training model 2/10 ---
  Window: 2023-10-26 ~ 2024-04-23 | Validation: 2024-04-23 ~ 2024-04-30
  Best iteration: 36, Best RMSE: 21.8798
--- Training model 3/10 ---
  Window: 2023-05-15 ~ 2023-11-11 | Validation: 2023-11-11 ~ 2023-11-18
  Best iteration: 7, Best RMSE: 15.4458
--- Training model 4/10 ---
  Window: 2023-04-10 ~ 2023-10-07 | Validation: 2023-10-07 ~ 2023-10-14
  Best iteration: 33, Best RMSE: 20.2108
--- Training model 5/10 ---
  Window: 2023-08-05 ~ 2024-02-01 | Validation: 2024-02-01 ~ 2024-02-08
  Best iteration: 97, Best RMSE: 17.4323
--- Training model 6/10 ---
  Window: 2023-02-18 ~ 2023-08-17 | Validation: 2023-08-17 ~ 2023-08-24
  Best iteration: 15, Best RMSE: 12.3572
--- Training model 7/10 ---
  Window: 2023-05-11 ~ 2023-11-07 | Validation: 2023-11-07 ~ 2023-11-14
  Best i

In [13]:
# ==============================================================================
# 6) 앙상블 모델을 이용한 재귀 예측
# ==============================================================================
print("\nRecursive prediction with ensemble model...")

def predict_with_ensemble(models, dmatrix):
    """앙상블 모델들의 예측값 평균을 반환합니다."""
    # 각 모델로부터 예측 수행
    predictions = [model.predict(dmatrix) for model in models]
    # 예측 결과들의 평균 계산
    return np.mean(predictions, axis=0)

all_preds = []
full_history = train.copy()

step_logs = []

# 기존 재귀 예측 로직과 거의 동일하나, final_model.predict -> predict_with_ensemble(models, ...)로 변경
for test_name, test_df in tests.items():
    test_df = test_df.copy()
    test_df["item_id"] = encoder.transform(test_df["영업장명_메뉴명"])
    test_df = make_date_feats(test_df)

    history = pd.concat([full_history, test_df], ignore_index=True)
    history = history.sort_values(["item_id", "영업일자"])

    last_date = test_df["영업일자"].max()
    items = test_df["영업장명_메뉴명"].unique()

    preds_rows = []
    current_date = last_date
    for step in range(1, 8):
        target_date = current_date + pd.Timedelta(days=1)
        frame = pd.DataFrame(
            {"영업일자": np.repeat(target_date, len(items)), "영업장명_메뉴명": items}
        )
        frame["item_id"] = encoder.transform(frame["영업장명_메뉴명"])
        frame = make_date_feats(frame)

        temp_hist = history.copy()
        for lag in [1, 7, 14, 28]:
            lagged = temp_hist[["영업일자", "item_id", "매출수량"]].copy()
            lagged["영업일자"] = lagged["영업일자"] + pd.Timedelta(days=lag)
            frame = frame.merge(
                lagged.rename(columns={"매출수량": f"lag_{lag}"}),
                on=["영업일자", "item_id"],
                how="left",
            )

        roll_base = temp_hist.sort_values(["item_id", "영업일자"]).copy()
        gb = roll_base.groupby("item_id")["매출수량"]
        roll_base["roll_mean_7"] = gb.rolling(7).mean().reset_index(0, drop=True)
        roll_base["roll_mean_14"] = gb.rolling(14).mean().reset_index(0, drop=True)
        roll_base["roll_std_7"] = gb.rolling(7).std().reset_index(0, drop=True)
        roll_base["영업일자"] = roll_base["영업일자"] + pd.Timedelta(days=1)
        frame = frame.merge(
            roll_base[
                ["영업일자", "item_id", "roll_mean_7", "roll_mean_14", "roll_std_7"]
            ],
            on=["영업일자", "item_id"],
            how="left",
        )

        nan_before = int(frame[feature_cols].isna().sum().sum())
        frame[feature_cols] = frame[feature_cols].fillna(0)
        nan_after = int(frame[feature_cols].isna().sum().sum())

        # 예측 수행 (앙상블)
        X_pred = frame[feature_cols]
        dpred = xgb.DMatrix(X_pred)
        # yhat = final_model.predict(dpred) # <- 기존 코드
        yhat = predict_with_ensemble(models, dpred) # <- 변경된 코드
        yhat = np.clip(yhat, 0, None)
        frame["pred"] = yhat

        pred_sum = float(yhat.sum())
        pred_mean = float(yhat.mean())
        pred_std = float(yhat.std() if len(yhat) else 0.0)
        print(
            f"[{test_name}] step {step} | date={target_date.date()} | "
            f"rows={len(frame):,} | NaN(before→after)={nan_before}->{nan_after} | "
            f"pred_sum={pred_sum:,.2f} | pred_mean={pred_mean:.3f} | pred_std={pred_std:.3f}"
        )

        add_hist = frame[["영업일자", "item_id", "영업장명_메뉴명", "pred"]].rename(
            columns={"pred": "매출수량"}
        )
        history = pd.concat([history, add_hist], ignore_index=True)

        frame_out = frame[["영업일자", "영업장명_메뉴명", "pred"]].copy()
        frame_out["영업일자"] = f"{test_name}+{step}일"
        preds_rows.append(frame_out)

        current_date = target_date

    test_pred = pd.concat(preds_rows, ignore_index=True)
    wide = test_pred.pivot(index="영업일자", columns="영업장명_메뉴명", values="pred")
    all_preds.append(wide)

print("\nPrediction complete")


Recursive prediction with ensemble model...
[TEST_00] step 1 | date=2024-07-14 | rows=193 | NaN(before→after)=0->0 | pred_sum=1,282.44 | pred_mean=6.645 | pred_std=13.919
[TEST_00] step 2 | date=2024-07-15 | rows=193 | NaN(before→after)=0->0 | pred_sum=880.16 | pred_mean=4.560 | pred_std=8.353
[TEST_00] step 3 | date=2024-07-16 | rows=193 | NaN(before→after)=0->0 | pred_sum=1,008.70 | pred_mean=5.226 | pred_std=12.343
[TEST_00] step 4 | date=2024-07-17 | rows=193 | NaN(before→after)=0->0 | pred_sum=1,045.61 | pred_mean=5.418 | pred_std=12.136
[TEST_00] step 5 | date=2024-07-18 | rows=193 | NaN(before→after)=0->0 | pred_sum=1,224.62 | pred_mean=6.345 | pred_std=13.684
[TEST_00] step 6 | date=2024-07-19 | rows=193 | NaN(before→after)=0->0 | pred_sum=1,359.11 | pred_mean=7.042 | pred_std=14.845
[TEST_00] step 7 | date=2024-07-20 | rows=193 | NaN(before→after)=0->0 | pred_sum=1,516.67 | pred_mean=7.858 | pred_std=12.774
[TEST_01] step 1 | date=2024-08-18 | rows=193 | NaN(before→after)=0->

In [14]:
# -----------------------------
# 0) 샘플 제출 파일 로드
# -----------------------------
sample = pd.read_csv("../open/sample_submission.csv")

# -----------------------------
# 7) 최종 제출 파일 생성
# -----------------------------
submission = pd.concat(all_preds)
submission = submission.reset_index().rename(columns={"index": "영업일자"})
submission = submission[sample.columns]   # sample과 같은 컬럼 순서로 맞춤
out_path = 'submission_xgboost_re.csv'
submission.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"Submission file created: {out_path}")


Submission file created: submission_xgboost_re.csv


In [15]:
import pandas as pd
import numpy as np
from IPython.display import display

# ==============================================================================
# 1. 설정 및 데이터 로드
# ==============================================================================
# [사용자 수정 필요] 파일 경로
#SUBMISSION_PATH = 'submission_wide_format.csv'
SUBMISSION_PATH = 'submission_xgboost_re.csv'
TRAIN_PATH = '../open/train/train.csv'

try:
    submission_df = pd.read_csv(SUBMISSION_PATH)
    train_df = pd.read_csv(TRAIN_PATH)
    print("✅ 평가 대상 파일(submission.csv, train.csv) 로드 완료.")
except FileNotFoundError:
    print("❌ 오류: 평가에 필요한 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
    submission_df, train_df = None, None

# ==============================================================================
# 2. 평가 함수 정의
# ==============================================================================
def weighted_smape(y_true, y_pred, weights):
    """
    가중치가 적용된 SMAPE를 계산하고 100점 만점으로 변환하는 함수.
    """
    # 실제값이 0인 데이터 필터링
    non_zero_mask = y_true != 0
    y_true_f = y_true[non_zero_mask]
    y_pred_f = y_pred[non_zero_mask]
    weights_f = weights[non_zero_mask]

    # SMAPE 계산
    numerator = np.abs(y_pred_f - y_true_f)
    denominator = (np.abs(y_true_f) + np.abs(y_pred_f))

    # 분모가 0이 되는 경우 방지 (y_true, y_pred 모두 0)
    # 위 필터로 y_true가 0인 경우는 제외되었으므로, y_pred만 0이어도 분모는 0이 아님
    element_wise_smape = 2 * numerator / denominator

    # 가중 평균 계산
    weighted_smape_score = np.sum(element_wise_smape * weights_f) / np.sum(weights_f)

    # 100점 만점 기준으로 변환
    return 100 * (1 - weighted_smape_score)

# ==============================================================================
# 3. 평가 실행 블록
# ==============================================================================
if submission_df is not None and train_df is not None:

    # --- 1. 데이터 전처리 ---
    # submission_df를 Long 포맷으로 변환
    pred_long = pd.melt(submission_df, id_vars=['영업일자'], var_name='영업장명_메뉴명', value_name='예측값')

    # train_df의 날짜 타입 변환
    train_df['영업일자'] = pd.to_datetime(train_df['영업일자'])

    # submission의 'TEST_XX+Y일'을 train의 실제 날짜와 매칭시키기 위한 가상 날짜 생성
    # (실제 평가에서는 이 부분이 주최측의 비공개 로직에 따라 달라짐)
    # 여기서는 train 데이터의 마지막 날짜 이후로 가정하여 매칭
    last_train_date = train_df['영업일자'].max()
    test_base_dates = {f'TEST_{i:02d}': last_train_date - pd.to_timedelta((9-i)*7 + 27, unit='d') for i in range(10)}

    def map_to_real_date(date_id):
        test_num = date_id.split('+')[0]
        day_offset = int(date_id.split('+')[1].replace('일', ''))
        return test_base_dates[test_num] + pd.to_timedelta(day_offset, unit='d')

    pred_long['실제영업일자'] = pred_long['영업일자'].apply(map_to_real_date)

    # --- 2. 실제값과 예측값 병합 ---
    eval_df = pd.merge(
        train_df,
        pred_long,
        left_on=['영업장명_메뉴명', '영업일자'],
        right_on=['영업장명_메뉴명', '실제영업일자'],
        how='inner' # 일치하는 날짜와 메뉴만 평가
    )

    if eval_df.empty:
        print("\n 경고: 예측 기간과 일치하는 실제값 데이터가 train.csv에 없습니다. 점수를 계산할 수 없습니다.")
    else:
        # --- 3. 가중치 부여 ---
        eval_df['영업장명'] = eval_df['영업장명_메뉴명'].str.split('_').str[0]
        # '담하', '미라시아'에 가중치 2, 나머지에 1 부여 (임의 설정)
        eval_df['가중치'] = np.where(eval_df['영업장명'].isin(['담하', '미라시아']), 2.0, 1.0)

        # --- 4. 점수 계산 ---
        y_true = eval_df['매출수량']
        y_pred = eval_df['예측값']
        weights = eval_df['가중치']

        # Public Score (50% 샘플링)
        public_sample = eval_df.sample(frac=0.5, random_state=42)
        public_score = weighted_smape(public_sample['매출수량'], public_sample['예측값'], public_sample['가중치'])

        # Private Score (100% 데이터)
        private_score = weighted_smape(y_true, y_pred, weights)

        print("\n" + "="*50)
        print("             리더보드 평가 점수 (모의 테스트)")
        print("="*50)
        print(f"Public Score  : {public_score:.4f}")
        print(f"Private Score : {private_score:.4f}")
        print("="*50)

✅ 평가 대상 파일(submission.csv, train.csv) 로드 완료.

             리더보드 평가 점수 (모의 테스트)
Public Score  : 32.1536
Private Score : 31.5636
